In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logreg').getOrCreate()

In [3]:
df = spark.read.csv('FileStore/tables/titanic.csv', inferSchema = True, header=True)

In [4]:
df.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [5]:
my_cols = df.select(['Survived', 
                     'Pclass',
                     'Sex',
                     'Age', 
                     'SibSp',
                     'Parch',
                     'Fare',
                     'Embarked'])

In [6]:
final_df = my_cols.na.drop()

In [7]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [8]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [9]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkedIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkedIndex', outputCol='EmbarkedVec')

In [10]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'EmbarkedVec', 'Age', 'SibSp', 'Parch','Fare'],
                            outputCol='features')

In [11]:
from pyspark.ml.classification import LogisticRegression

In [12]:
from pyspark.ml import Pipeline

In [13]:
log_reg = LogisticRegression(featuresCol='features', labelCol='Survived')

In [14]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer,
                            gender_encoder, embark_encoder,
                            assembler, log_reg])

In [15]:
train_data, test_data = final_df.randomSplit([0.7,0.3])

In [16]:
model = pipeline.fit(train_data)

In [17]:
results = model.transform(test_data)

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [19]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [20]:
auc = my_eval.evaluate(results)

In [21]:
auc

Out[ 21 ]: 0.7995873857942822